# Project #2 Artificial intelligance(Game)
# (spring 2022)

    Amirmahdi Ansaripour

    Student ID: 810198358

Approach: 

Completing implementation of a game in a way that we always have a winning strategy against each rival's action. 
   
Description:

The incomplete code is given and a winning strategy should be added in form of a minimax algorithm. The algorithm simply creates all possible states per action and chooses the best one based on a fitness property each state has. 

The approach takes alot of time due to considering each state and calculating it's fitness. So  we add alpha-beta technique to reduce time.

### Minimax implementation  

minimax(self, maxorMin, depth, prune, alpha, beta)

Here, minimax gets 5 arguments:

maxorMin: whether the current node is max or min, in other words, is it our turn or the opponent's turn?

depth: Current depth of tree

prune: A boolean variable indicating that pruning is used or not

alpha, beta: Pruning arguments (used if prune == True)

### Heuristic function

When using minimax, most of the times we don't have time to get deep into the leaves. So, we should cut the procedure at lower heights and consider middle nodes as leaves. That's the reason of using heuristics in minimax.

The heuristic I've defined is based on the number of consecutive same-colored peices that aren't blocked at their ends. and the higher the size of consecutive section, the higher point it gets.Being consecutive is measured row-wise, column-wise, and diameter-wise.

In [142]:
from random import random
import copy
import math
import time
DEPTH = 1
PRUNE = False
numberofSeenNodes = 0
numofImplementation = 200
board_sizes_to_check = [(6,7), (7,8), (7,10)]

DepthsforWithoutPruning = [1, 3, 5]
DepthsforPruning = [1, 3, 5, 7]

class ConnectSin:
    YOU = 1
    CPU = -1
    EMPTY = 0
    DRAW = 0
    __CONNECT_NUMBER = 4
    board = None

    def __init__(self, board_size=(6, 7), silent=False):
        """
        The main class for the connect4 game

        Inputs
        ----------
        board_size : a tuple representing the board size in format: (rows, columns)
        silent     : whether the game prints outputs or not
        """
        assert len(board_size) == 2, "board size should be a 1*2 tuple"
        assert board_size[0] > 4 and board_size[1] > 4, "board size should be at least 5*5"

        self.columns = board_size[1]
        self.rows = board_size[0]
        self.silent = silent
        self.board_size = self.rows * self.columns

    def run(self, starter=None):
        """
        runs the game!

        Inputs
        ----------
        starter : either -1,1 or None. -1 if cpu starts the game, 1 if you start the game. None if you want the starter
            to be assigned randomly 

        Output
        ----------
        (int) either 1,0,-1. 1 meaning you have won, -1 meaning the player has won and 0 means that the game has drawn
        """
        if (not starter):
            starter = self.__get_random_starter()
        assert starter in [self.YOU, self.CPU], "starter value can only be 1,-1 or None"
        
        self.__init_board()
        turns_played = 0
        current_player = starter
        while(turns_played < self.board_size):
            
            if (current_player == self.YOU):
                self.__print_board()
                player_input = self.get_your_input()
            elif (current_player == self.CPU):
                player_input = self.__get_cpu_input()
            else:
                raise Exception("A problem has happend! contact no one, there is no fix!")
            if (not self.register_input(player_input, current_player)):
                self.__print("this move is invalid!")
                continue
            current_player = self.__change_turn(current_player)
            potential_winner = self.check_for_winners()
            turns_played += 1
            if (potential_winner != 0):
                self.__print_board()
                self.__print_winner_message(potential_winner)
                return potential_winner
        self.__print_board()
        self.__print("The game has ended in a draw!")
        return self.DRAW

    def get_your_input(self):
        """
        gets your input

        Output
        ----------
        (int) an integer between 1 and column count. the column to put a piece in
        """

        choice = self.minimaxWithPruning(True, DEPTH, -2e9, 2e9)
        return choice[1]
        raise NotImplementedError

    def check_for_winners(self):
        """
        checks if anyone has won in this position

        Output
        ----------
        (int) either 1,0,-1. 1 meaning you have won, -1 meaning the player has won and 0 means that nothing has happened
        """
        have_you_won = self.check_if_player_has_won(self.YOU)
        if have_you_won:
            return self.YOU
        has_cpu_won = self.check_if_player_has_won(self.CPU)
        if has_cpu_won:
            return self.CPU
        return self.EMPTY

    def check_if_player_has_won(self, player_id):
        """
        checks if player with player_id has won

        Inputs
        ----------
        player_id : the id for the player to check

        Output
        ----------
        (boolean) true if the player has won in this position
        """
        return (
            self.__has_player_won_diagonally(player_id)
            or self.__has_player_won_horizentally(player_id)
            or self.__has_player_won_vertically(player_id)
        )
    
    def is_move_valid(self, move):
        """
        checks if this move can be played

        Inputs
        ----------
        move : the column to place a piece in, in range [1, column count]

        Output
        ----------
        (boolean) true if the move can be played
        """
        if (move < 1 or move > self.columns):
            return False
        column_index = move - 1
        return self.board[0][column_index] == 0
    
    def get_possible_moves(self):
        """
        returns a list of possible moves for the next move

        Output
        ----------
        (list) a list of numbers of columns that a piece can be placed in
        """
        possible_moves = []
        for i in range(self.columns):
            move = i + 1
            if (self.is_move_valid(move)):
                possible_moves.append(move)
        return possible_moves
    
    def register_input(self, player_input, current_player):
        """
        registers move to board, remember that this function changes the board

        Inputs
        ----------
        player_input : the column to place a piece in, in range [1, column count]
        current_player: ID of the current player, either self.YOU or self.CPU

        """
        if (not self.is_move_valid(player_input)):
            return False
        self.__drop_piece_in_column(player_input, current_player)
        return True

    def __init_board(self):
        self.board = []
        for i in range(self.rows):
            self.board.append([self.EMPTY] * self.columns)

    def __print(self, message: str):
        if not self.silent:
            print(message)

    def __has_player_won_horizentally(self, player_id):
        for i in range(self.rows):
            for j in range(self.columns - self.__CONNECT_NUMBER + 1):
                has_won = True
                for x in range(self.__CONNECT_NUMBER):
                    if self.board[i][j + x] != player_id:
                        has_won = False
                        break
                if has_won:
                    return True
        return False

    def __has_player_won_vertically(self, player_id):
        for i in range(self.rows - self.__CONNECT_NUMBER + 1):
            for j in range(self.columns):
                has_won = True
                for x in range(self.__CONNECT_NUMBER):
                    if self.board[i + x][j] != player_id:
                        has_won = False
                        break
                if has_won:
                    return True
        return False

    def __has_player_won_diagonally(self, player_id):
        for i in range(self.rows - self.__CONNECT_NUMBER + 1):
            for j in range(self.columns - self.__CONNECT_NUMBER + 1):
                has_won = True
                for x in range(self.__CONNECT_NUMBER):
                    if self.board[i + x][j + x] != player_id:
                        has_won = False
                        break
                if has_won:
                    return True
                has_won = True
                for x in range(self.__CONNECT_NUMBER):
                    if self.board[i + self.__CONNECT_NUMBER - 1 - x][j + x] != player_id:
                        has_won = False
                        break
                if has_won:
                    return True
        return False
    
#     def diagnolly(self):
#         maxDia = 0
#         for i in range(self.rows - self.__CONNECT_NUMBER + 1):
#             for j in range(self.columns - self.__CONNECT_NUMBER + 1):
#                 total = self.board[i][j] + self.board[i + 1][j + 1] + self.board[i + 2][j + 2]
#                 total += self.board[i + self.__CONNECT_NUMBER - 1][j] + self.board[i + self.__CONNECT_NUMBER - 2][j + 1] + self.board[i + self.__CONNECT_NUMBER - 3][j + 2]
#                 maxDia += total
#         return maxDia
    
    def __get_random_starter(self):
        players = [self.YOU, self.CPU]
        return players[int(random() * len(players))]
    
    def __get_cpu_input(self):
        """
        This is where clean code goes to die.
        """
        bb = copy.deepcopy(self.board)
        pm = self.get_possible_moves()
        for m in pm:
            self.register_input(m, self.CPU)
            if (self.check_if_player_has_won(self.CPU)):
                self.board = bb
                return m
            self.board = copy.deepcopy(bb)
        if (self.is_move_valid((self.columns // 2) + 1)):
            c = 0
            cl = (self.columns // 2) + 1
            for x in range(self.rows):
                if (self.board[x][cl] == self.CPU):
                    c += 1
            if (random() < 0.65):
                return cl
        return pm[int(random() * len(pm))]
    
    def __drop_piece_in_column(self, move, current_player):
        last_empty_space = 0
        column_index = move - 1
        for i in range(self.rows):
            if (self.board[i][column_index] == 0):
                last_empty_space = i
        self.board[last_empty_space][column_index] = current_player
        return True
        
    def __print_winner_message(self, winner):
        if (winner == self.YOU):
            self.__print("congrats! you have won!")
        else:
            self.__print("gg. CPU has won!")
    
    def __change_turn(self, turn):
        if (turn == self.YOU): 
            return self.CPU
        else:
            return self.YOU

    def __print_board(self):
        if (self.silent): return
        print("Y: you, C: CPU")
        for i in range(self.rows):
            for j in range(self.columns):
                house_char = "O"
                if (self.board[i][j] == self.YOU):
                    house_char = "Y"
                elif (self.board[i][j] == self.CPU):
                    house_char = "C"
                    
                print(f"{house_char}", end=" ")
            print()
        
#     def horizentally(self):
#         maxRow = 0
#         for i in range(self.rows):
#             j = 0
#             while(j < self.columns - self.__CONNECT_NUMBER + 1):
#                 total = self.board[i][j + 2] + self.board[i][j + 1] + self.board[i][j]
#                 maxRow += total
#                 j += 1
#         return maxRow
    def horizentally(self, sign):
        maxRow = 0
        for i in range(self.rows):
            j = 0
            while(j < self.columns):
                if self.board[i][j] == sign:
                    if j + 1 < self.columns and self.board[i][j + 1] == sign: 
                        maxRow += 2
                        if j + 2 < self.columns and self.board[i][j + 2] == sign:
                            maxRow += 18
                            if(j >= 1 and self.board[i][j - 1] == self.EMPTY) or (j + 3 < self.columns and self.board[i][j + 3] == self.EMPTY):
                                maxRow += 20
                j += 1
        return maxRow

    def vertically(self, sign):
        maxCol = 0
        for j in range(self.columns):
            i = 0
            while(i < self.rows):
                if self.board[i][j] == sign:
                   if i + 1 < self.rows and self.board[i + 1][j] == sign:
                        maxCol += 2
                        if i + 2 < self.rows and self.board[i + 2][j] == sign:
                            maxCol += 18
                            if (i >= 1 and self.board[i - 1][j] == self.EMPTY) or (i + 3 < self.rows and self.board[i + 3][j] == self.EMPTY):
                                maxCol += 80

                i += 1

        return maxCol
    
    def diagnolly(self, sign):
        maxDia = 0
        for i in range(self.rows - self.__CONNECT_NUMBER + 1):
            for j in range(self.columns - self.__CONNECT_NUMBER + 1):
                has_won = 0
                for x in range(self.__CONNECT_NUMBER):
                    if self.board[i + x][j + x] == sign:
                        has_won += 1
                        if has_won == 1:
                            maxDia += 2
                        elif has_won == 2:
                            maxDia += 20

                if has_won == 3:
                    if (i >= 1 and j >= 1 and self.board[i - 1][j - 1] == self.EMPTY) or (i + x < self.rows and j + x < self.columns and self.board[i + x][j + x] == self.EMPTY):
                        maxDia += 100

                
                has_won = 0
                for x in range(self.__CONNECT_NUMBER):
                    if self.board[i + self.__CONNECT_NUMBER - 1 - x][j + x] == sign:
                        has_won += 1
                        if has_won == 1:
                            maxDia += 2
                        if has_won == 2:
                            maxDia += 20

                if has_won == 3:
                    if(i + self.__CONNECT_NUMBER - 1 - x >= 0 and j + x < self.columns and self.board[i + self.__CONNECT_NUMBER - 1 - x][j + x] == self.EMPTY) or (i + self.__CONNECT_NUMBER < self.rows and j - 1 >= 0 and self.board[i + self.__CONNECT_NUMBER][j - 1] == self.EMPTY):
                        maxDia += 100
                    
        return maxDia


#     def vertically(self):
#         maxCol = 0
#         for j in range(self.columns):
#             i = 0
#             while(i < self.rows - self.__CONNECT_NUMBER + 1):
#                 total = self.board[i][j] + self.board[i + 1][j] + self.board[i + 2][j]
#                 maxCol += total
#                 i += 1

#         return maxCol
    
    def deepCopy(self):
        copied = ConnectSin((self.rows, self.columns), self.silent)
        copied.board = [list(x) for x in self.board]
        copied.YOU = 1
        copied.CPU = -1
        return copied
    
    def heuristic(self):
        return self.check_for_winners()
#         rowYOU = self.vertically(self.YOU)
#         colYOU = self.horizentally(self.YOU)
#         diaYOU = self.diagnolly(self.YOU)
#         pYOU = rowYOU + colYOU + diaYOU
        

#         rowCPU = self.vertically(self.CPU)
#         colCPU = self.horizentally(self.CPU)
#         diaCPU = self.diagnolly(self.CPU) 
        
#         disadvantage = rowCPU + colCPU + diaCPU

#         pYOU -= disadvantage

#         if(self.check_for_winners() == self.YOU): pYOU = math.inf
#         elif (self.check_for_winners() == self.CPU): pYOU = -1 * math.inf

#         return pYOU

    def minimaxWithPruning(self, maxorMin, depth, alpha, beta):
        possibleMoves = self.get_possible_moves()
        if(depth <= 0):    
            h = self.heuristic()
            return (h, math.inf)
        global numberofSeenNodes 
        numberofSeenNodes += 1
        chooseBetween = []

        maxx = 2e9
        minn = -2e9

        for column in possibleMoves:
            child = self.deepCopy()
            if(maxorMin == True):
                child.register_input(column, child.YOU)
            else:
                child.register_input(column, child.CPU)
        
            answer = child.minimaxWithPruning(1 - maxorMin, depth - 1, alpha, beta) ##answer = (cost, column)
            if(PRUNE):
                if (maxorMin == True):
                    maxx = max(maxx, answer[0])
                    if (maxx >= beta):
                        return (answer[0], column)
                    alpha = max(alpha, maxx)
                else:
                    minn = min(minn, answer[0])
                    if (minn <= alpha):
                        return (answer[0], column)
                    beta = min(beta, minn)
                
            chooseBetween.append((answer[0], column))
        chooseBetween.sort(key = lambda t: t[0], reverse= maxorMin)
        return chooseBetween[0]

        

In [143]:
def test(depths, board_sizes_to_check):
    total = numofImplementation
    winnigProbability = [[0 for jj in range(0, len(board_sizes_to_check))] for ii in range(0, len(depths))]
    timeofEachDepth = [[0 for jj in range(0, len(board_sizes_to_check))] for ii in range(0, len(depths))]
    seenofEachDepth = [[0 for jj in range(0, len(board_sizes_to_check))] for ii in range(0, len(depths))]
    for i in range(0, len(depths)):
        global DEPTH
        DEPTH = depths[i]
        for j in range(0,len(board_sizes_to_check)):
            print("DEPTH: ", depths[i], " BOARD SIZE: ", board_sizes_to_check[j], " PRUNE: ", PRUNE)
            for k in range(0, numofImplementation):
                global numberofSeenNodes
                numberofSeenNodes = 0
                start = time.time()
                game = ConnectSin(board_sizes_to_check[j], True)
                winner = game.run()
                finish = time.time()
                if winner == 1:
                    winnigProbability[i][j] += 1
                seenofEachDepth[i][j] += numberofSeenNodes
                timeofEachDepth[i][j] += (finish - start)    
            print("Winning probability: ", winnigProbability[i][j] / total)
            print("Time: ", timeofEachDepth[i][j] / total)
            print("Seen nodes: ", math.floor(seenofEachDepth[i][j] / total), "\n")
        print("\n\n\n")
            
global PRUNE
PRUNE = True
test(DepthsforPruning, board_sizes_to_check)
global PRUNE
PRUNE = False        
test(DepthsforWithoutPruning, board_sizes_to_check)


DEPTH:  1  BOARD SIZE:  (6, 7)  PRUNE:  True
Winning probability:  0.71
Time:  0.0011533522605895997
Seen nodes:  3 

DEPTH:  1  BOARD SIZE:  (7, 8)  PRUNE:  True
Winning probability:  0.685
Time:  0.0018672537803649902
Seen nodes:  3 

DEPTH:  1  BOARD SIZE:  (7, 10)  PRUNE:  True
Winning probability:  0.74
Time:  0.0025671112537384035
Seen nodes:  3 





DEPTH:  3  BOARD SIZE:  (6, 7)  PRUNE:  True
Winning probability:  0.635
Time:  0.0012181401252746582
Seen nodes:  11 

DEPTH:  3  BOARD SIZE:  (7, 8)  PRUNE:  True
Winning probability:  0.76
Time:  0.0021726810932159424
Seen nodes:  11 

DEPTH:  3  BOARD SIZE:  (7, 10)  PRUNE:  True
Winning probability:  0.78
Time:  0.0031093335151672363
Seen nodes:  11 





DEPTH:  5  BOARD SIZE:  (6, 7)  PRUNE:  True
Winning probability:  0.64
Time:  0.001287006139755249
Seen nodes:  19 

DEPTH:  5  BOARD SIZE:  (7, 8)  PRUNE:  True
Winning probability:  0.72
Time:  0.0018180382251739501
Seen nodes:  19 

DEPTH:  5  BOARD SIZE:  (7, 10)  PRUNE: 

Without Pruning

| [depth, table size] |  Average Seen nodes | Average Time | Winnig chance |
| --- | --- | --- | --- |
|  [1, (6,7)]| 3 | 0.001s | 0.675 |
|  [1, (7,8)] | 4 | 0.003s | 0.775 |
|  [1,(7,10)] | 3 | 0.006s | 0.72 |
|  [3, (6,7)]| 243 | 0.056s | 0.715 |
|  [3, (7,8)] | 296 | 0.109s | 0.735 |
|  [3,(7,10)] | 446 | 0.300s | 0.79 |
|  [5, (6,7)]| 11863 | 2.692s | 0.78 |
|  [5, (7,8)] | 19700 | 7.254s | 0.735 |
|  [5,(7,10)] | 44769 | 26.298s | 0.785 |

With pruning 

| [depth, table size] | Average Seen nodes | Average Time | Winnig chance |
| --- | --- | --- | --- |
|  [1, (6,7)]| 3 | 0.001s | 0.71 |
|  [1, (7,8)] | 3 | 0.001s | 0.685 |
|  [1,(7,10)] | 3 | 0.002s | 0.74 |
|  [3, (6,7)]| 11 | 0.001s | 0.635 |
|  [3, (7,8)] | 11 | 0.002s | 0.76 |
|  [3,(7,10)] | 11 | 0.003s | 0.78 |
|  [5, (6,7)]| 19 | 0.001s | 0.64 |
|  [5, (7,8)] | 19 | 0.001s | 0.72 |
|  [5,(7,10)] | 19 | 0.002s | 0.715 |
|  [7, (6,7)]| 28 | 0.001s | 0.72 |
|  [7, (7,8)] | 27 | 0.001s | 0.655 |
|  [7,(7,10)] | 27 | 0.002s | 0.745 |


### Questions

    1) What properties does a good heuristic have? Explain the reason you had for choosing your heuristic, and yours advantages over other heuristics.

A good heuristic should estimate the current node's fitness accurately (The node is not a leaf mostly, because leaves' fitnesses are either +inf (we won) of -inf (opponent won)).

My heuristic, as explained earlier, depends on the number of consecutive 1's and -1's. Of course it depends on the fact that are consecutive elements blocked or not, either(See the fitness function). It imitates the real world with a good approximation, since in the real game, the higher a state has unblocked consecutive peices, the more it's critical and needs attention.

    2) Do you find a relation between the depth and demanded parameters? Explain how the depth and number of seen nodes, time, and winning chance relate to each other?

Without any doubt, number of seen nodes and time have a direct relation with depth.

But the winning chance isn't influenced by depth. The only thing to mention is that the more the algorithm gets deeper, the less it depends on the heuristic since it's getting closer and closer to the real answers.

    3) When pruning, in what order did you add the childs for each node? Does the order matter? Explain the reason of adding childs to nodes in that order.

At first, the order really matters in pruning and if we use the opposite order, it won't be different from minimax without pruning.

The important point is: for each min or max node, the more a child is favorable, the more it should be left-sided. 

So in each stage(node), we should sort our choices. If the node is min-type, sorting has ascending order, if it's max, descending order. This helps us alot in time complexity and omits alot of subtrees and nodes.  

### Conclusion
We've already learnt the concept of minimax algorithm and how they help us against a rival(probably rational) in games. 

Besides, We've found out that this algorithm takes a lot of time when the depth gets higher. So we used another technique named Alpha Beta pruning which is useful on some conditions metioned in Q3. 

### Refrences

https://ai.berkeley.edu/~cs188/sp12/slides/cs188%20lecture%206%20and%207%20--%20games%20search%206PP.pdf

https://www.geeksforgeeks.org/minimax-algorithm-in-game-theory-set-1-introduction/

https://www.geeksforgeeks.org/minimax-algorithm-in-game-theory-set-4-alpha-beta-pruning/